## 7.2 Autoencoder

In [5]:
import numpy as np
import tensorflow as tf
from sklearn import datasets

class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
            
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x:[data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, "./model.ckpt")
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x:data})
            print('input',data)
            print('compressed', hidden)
            print('reconstructed', reconstructed)
            return reconstructed
        
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 4.53341150284
epoch 10: loss = 2.17578554153
epoch 20: loss = 1.18200922012
epoch 30: loss = 0.668567299843
epoch 40: loss = 0.418907284737
epoch 50: loss = 0.388832002878
epoch 60: loss = 0.383747577667
epoch 70: loss = 0.37843900919
epoch 80: loss = 0.369272381067
epoch 90: loss = 0.354299098253
epoch 100: loss = 0.334111690521
epoch 110: loss = 0.314820200205
epoch 120: loss = 0.29385098815
epoch 130: loss = 0.26940742135
epoch 140: loss = 0.250785768032
epoch 150: loss = 0.228750839829
epoch 160: loss = 0.213939428329
epoch 170: loss = 0.213330283761
epoch 180: loss = 0.215298101306
epoch 190: loss = 0.216015771031
epoch 200: loss = 0.21477355063
epoch 210: loss = 0.214092597365
epoch 220: loss = 0.214185386896
epoch 230: loss = 0.214667573571
epoch 240: loss = 0.215232625604
('input', [[8, 4, 6, 2]])
('compressed', array([[-0.63872153]], dtype=float32))
('reconstructed', array([[ 6.45275974,  2.85445881,  5.3974576 ,  1.90080643]], dtype=float32))


array([[ 6.45275974,  2.85445881,  5.3974576 ,  1.90080643]], dtype=float32)

## 7.3 Batch training

* 시간에 대한 압력이 없다면, 네트워크를 하나씩 훈련하는 것이 가장 안전한 방법이다.
* 하지만 네트워크가 원하는 시간보다 오래 걸린다면, 입력으로 한번에 여러 데이터를 넣고 훈련하는 것이 좋은 해결책이며 이를 batch training이라고 부른다.
* 일반적으로 batch size가 증가되면 알고리즘의 속도는 빨라지지만, 성공적인 수렴은 거의 보장되지 않는다.
* 양날의 칼과도 같기 때문에 적절한 사용이 필요하다.


In [8]:
import numpy as np
import tensorflow as tf
from sklearn import datasets

class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, batch_size=10, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
            
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()

    def get_batch(X, size):
        a = np.random.choice(len(X), size, replace=False)
        return X[a]

    def train(self, data):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(500):
                batch_data = get_batch(data, self.batch_size)
                l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x:batch_data})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, "./model.ckpt")
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x:data})
            print('input',data)
            print('compressed', hidden)
            print('reconstructed', reconstructed)
            return reconstructed
        
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 3.52132701874
epoch 10: loss = 3.4291934967
epoch 20: loss = 4.0539894104
epoch 30: loss = 3.65123748779
epoch 40: loss = 3.64075255394
epoch 50: loss = 3.66780972481
epoch 60: loss = 3.89273262024
epoch 70: loss = 3.72357821465
epoch 80: loss = 3.45523881912
epoch 90: loss = 3.41217517853
epoch 100: loss = 3.88508605957
epoch 110: loss = 3.63075590134
epoch 120: loss = 3.20593881607
epoch 130: loss = 3.90278840065
epoch 140: loss = 3.59414052963
epoch 150: loss = 3.6022670269
epoch 160: loss = 3.67340016365
epoch 170: loss = 3.19579434395
epoch 180: loss = 3.24538803101
epoch 190: loss = 3.38957095146
epoch 200: loss = 3.44335532188
epoch 210: loss = 3.75995755196
epoch 220: loss = 3.2501282692
epoch 230: loss = 3.95871853828
epoch 240: loss = 3.28261899948
epoch 250: loss = 3.25385737419
epoch 260: loss = 3.38012862206
epoch 270: loss = 3.1948621273
epoch 280: loss = 3.22942376137
epoch 290: loss = 3.02328872681
epoch 300: loss = 3.59085869789
epoch 310: loss = 2.9752

array([[ 1.27258229,  0.7419734 ,  1.94552088,  0.46711466]], dtype=float32)